In [3]:
import pandas as pd
import os

cad = pd.read_csv("dataset/cadastro-de-aerodromos-civis-publicos.csv", sep=";", dtype=str)
print(cad.columns)

Index(['CÓDIGO OACI', 'CIAD', 'NOME', 'MUNICÍPIO ATENDIDO', 'UF', 'LATITUDE',
       'LONGITUDE', 'ALTITUDE', 'OPERAÇÃO', 'OACI', 'AIP BRASIL', 'DESIGNAÇÃO',
       'COMPRIMENTO', 'LARGURA', 'RESISTÊNCIA', 'SUPERFÍCIE', 'DESIGNAÇÃO.1',
       'COMPRIMENTO.1', 'LARGURA.1', 'RESISTÊNCIA.1', 'SUPERFÍCIE.1',
       'DESIGNAÇÃO.2', 'COMPRIMENTO.2', 'LARGURA.2', 'RESISTÊNCIA.2',
       'SUPERFÍCIE.2', 'RAMPA(S) DE APROXIMAÇÃO', 'FORMATO DA ÁREA DE POUSO',
       'DIMENSÕES', 'RESISTÊNCIA.3', 'SUPERFÍCIE.3', 'p1', 'p2', 'p3', 'p4',
       'PORTARIAS', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'PORTARIA Nº 16.739/2025 e alterações',
       'PORTARIA Nº 11.183/2023\nA Classe I se divide ainda em RBAC 121, RBAC 135 Regular e Outras Operações',
       'PORTARIA Nº 2007/2014 E ALTERAÇÕES\n(* = REGIME TARIFÁRIO DIFERENCIADO)',
       'AERONAVE CRÍTICA', 'TIPO DE APROX. (CABECEIRAS)', 'FREQ. SEMANAL',
       'r1', 'r2', 'REFERÊNCIA(S)', 'RESTRIÇÃO(ÕES)', 'SITUAÇÃO', 'r1.1',
       'REFERÊN

In [ ]:
import pandas as pd
import os

# ============================
# 1. CARREGAR MAPA ICAO → CIDADE
# ============================

cadastro_path = "dataset/cadastro-de-aerodromos-civis-publicos.csv"

cad = pd.read_csv(cadastro_path, sep=";", dtype=str)

# Normalizar nomes
cad["CÓDIGO OACI"] = cad["CÓDIGO OACI"].str.upper().str.strip()
cad["MUNICÍPIO ATENDIDO"] = cad["MUNICÍPIO ATENDIDO"].str.upper().str.strip()

icao_to_city = dict(zip(cad["CÓDIGO OACI"], cad["MUNICÍPIO ATENDIDO"]))

print("✔️ Mapa ICAO → Cidade carregado:", len(icao_to_city), "aeroportos")


# ============================
# 2. UNIFICAR OS CSVs DOS ANOS
# ============================

base_folder_path = "./dataset/"
df_list = []

# pastas 2023, 2024, 2025...
year_folders = [
    d for d in os.listdir(base_folder_path)
    if os.path.isdir(os.path.join(base_folder_path, d)) and d.isdigit()
]

for year_folder in year_folders:
    year_path = os.path.join(base_folder_path, year_folder)
    year = int(year_folder)

    for csv_file in os.listdir(year_path):
        if not csv_file.lower().endswith(".csv"):
            continue

        file_path = os.path.join(year_path, csv_file)

        try:
            df = pd.read_csv(file_path, sep=";", dtype=str)

            # normalizar ICAO
            df["ORIGEM"] = df["ORIGEM"].str.upper().str.strip()
            df["DESTINO"] = df["DESTINO"].str.upper().str.strip()

            # ano garantido
            df["ANO"] = year

            # MES normalizado
            df["MES"] = df["MES"].str.replace(",", "").astype(int)

            # TARIFA numérica
            df["TARIFA"] = df["TARIFA"].str.replace(",", ".").astype(float)

            # criar colunas de cidade
            df["CIDADE_ORIGEM"] = df["ORIGEM"].map(icao_to_city)
            df["CIDADE_DESTINO"] = df["DESTINO"].map(icao_to_city)

            df_list.append(df)

        except Exception as e:
            print(f"❌ ERRO ao ler {file_path}: {e}")

# ============================
# 3. CONCATENAR
# ============================

main_df = pd.concat(df_list, ignore_index=True)

print("\n✔️ DataFrame final criado!")
print(main_df.head(20))

# ============================
# 4. EXPORTAR
# ============================

out = "dataset_unificado.csv"
main_df.to_csv(out, sep=";", index=False, encoding="utf-8")
print("\n📦 Exportado:", out)


✔️ Mapa ICAO → Cidade carregado: 504 aeroportos
